<a href="https://colab.research.google.com/github/joselufb/03MAIR-Algoritmos-Optimizacion/blob/main/Algoritmos_AG2_Jose_Luis_Fernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos - Actividad Guiada 2

Nombre: José Luis Fernández Bastida

URL: https://colab.research.google.com/drive/1NZt8mMOPp7hid9A09srmWCJlmHDrzQjm?usp=sharing

GitHub: https://github.com/joselufb/03MAIR-Algoritmos-Optimizacion